In [1]:
import torch
from classes import FullFileForTestings  # Import dataset class and model
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter


class SpeedEstimatorRNNModified(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SpeedEstimatorRNNModified, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected layer

    def forward(self, x):
        out, _ = self.rnn(x)  # RNN returns (output, hidden_state)
        out = self.fc(out)  # Apply the fully connected layer to all time steps
        return out

In [2]:
#Hyperparameters
input_size = 14  # Number of CAN signals per timestep
hidden_size = [64, 64, 64, 128, 128, 256, 128, 128, 128, 128, 128, 128]
num_layers = [1, 2, 3, 1, 3, 3, 2, 2, 2, 2, 4, 3]
output_size = 1
learning_rate = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.005, 0.001, 0.001, 0.0001, 0.0005, 0.001]
num_epochs = 15

In [8]:
num_models = 12
num_test_data_path = [r"C:\my files\thesis\AI_training\data\testing_data\testing_data_1\test_auax\auax_drv_30.csv", r"C:\my files\thesis\AI_training\data\testing_data\testing_data_1\test_buax\buax_brk_30.csv", r"C:\my files\thesis\AI_training\data\testing_data\testing_data_1\test_sin_steer\sin_steer_30.csv", r"C:\my files\thesis\AI_training\data\testing_data\testing_data_1\test_suay\suay_mid_99.csv"]
test_data = ["auax_drv", "buax_brk", "sin_steer", "suay_mid"]
model_location = r"C:\my files\thesis\AI_training\trained_models\iteration_2\model_"

for i in range(num_models):
    # Set paths and parameters for testing
    model_path = model_location + str(i)  # Replace with the path to your saved model

    for j in range(len(num_test_data_path)):
        test_csv_path = num_test_data_path[j]
        # Prepare the test dataset
        test_dataset = FullFileForTestings(test_csv_path)
        features, actual_speeds = test_dataset.get_full_data()

        # Pass all data to the model directly
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        features = features.unsqueeze(0)  # Add batch dimension
        features = features.to(device)

        # Define and load the model
        model = SpeedEstimatorRNNModified(input_size, hidden_size[i], num_layers[i], output_size)  # Instantiate your RNN model
        checkpoint = torch.load(model_path, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])  # Load the trained weights
        model = model.to(device)  # Move model to the appropriate device
        model.eval()  # Set model to evaluation mode

        # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds.numpy(), label="Actual Speeds")
        plt.plot(predicted_speeds, label="Predicted Speeds", linestyle="dashed")
        plt.legend()
        plt.title("Comparison of Actual vs. Predicted Vehicle Speeds (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed [m/s]")
        plt.grid()
        plt.savefig(r"C:\my files\thesis\AI_training\testing_graphs\iteration_2\model_" + str(i) + "\model_" + str(i) + "_" +test_data[j] + "_act_vs_predicted_speed.png", dpi=300,)
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds.numpy()-predicted_speeds, label="Difference")
        plt.legend()
        plt.title("Difference of Actual vs. Predicted Vehicle Speeds (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed difference [m/s]")
        plt.grid()
        plt.savefig(r"C:\my files\thesis\AI_training\testing_graphs\iteration_2\model_" + str(i) + "\model_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_speed_diff.png", dpi=300)
        plt.close()

<>:42: SyntaxWarning: invalid escape sequence '\m'
<>:52: SyntaxWarning: invalid escape sequence '\m'
<>:42: SyntaxWarning: invalid escape sequence '\m'
<>:52: SyntaxWarning: invalid escape sequence '\m'
C:\Users\ROG\AppData\Local\Temp\ipykernel_22888\879203956.py:42: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(r"C:\my files\thesis\AI_training\testing_graphs\iteration_2\model_" + str(i) + "\model_" + str(i) + "_" +test_data[j] + "_act_vs_predicted_speed.png", dpi=300,)
C:\Users\ROG\AppData\Local\Temp\ipykernel_22888\879203956.py:52: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(r"C:\my files\thesis\AI_training\testing_graphs\iteration_2\model_" + str(i) + "\model_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_speed_diff.png", dpi=300)


In [5]:
import os

# Define the path where you want to create the directory
directory_path = r"C:\my files\thesis\AI_training\testing_graphs\iteration_2\model_"

for k in range(12):
    # Create the directory
    directory = directory_path + str(k)
    os.makedirs(directory, exist_ok=True)
